This will be something of an initial look at making predictions on flight delays.

We can use a basic linear regression initially

In [1]:
import numpy as np
import pandas as pd
import pgaccess as pg

from sklearn.model_selection import train_test_split

The columns available are:
* Date
* Carrier
* Tail Number
* Origin
* Destination
* Scheduled Departure Time
* Scheduled Arrival Time
* Number of Flights ???
* Distance

I'll start with the average delay on a route, the average delay for a carrier in that month, and plane speed for a given tail number.  
The important features to take from this list will be the scheduled departure time, scheduled elapsed time, and distance. I don't think both arrival and departure time will be needed.

Lets load the pre-prepared data

In [2]:
# Average delay on a route
routeDelay = pd.read_csv('../data/delays_per_airport_pair.csv', index_col=0)
routeDelay.fillna(0, inplace=True)
# Monthly carrier delay
carrierDelay = pd.read_csv('../data/monthly_carrier_delay.csv')
carrierDelay.set_index(['op_unique_carrier', 'month'], inplace=True)
# Plane Speed
planeSpeed = pd.read_csv('../data/plane_speed.csv')
planeSpeed.set_index(['tail_num'], inplace=True)

routeDelay.shape, carrierDelay.shape, planeSpeed.shape

((372, 372), (320, 2), (6481, 4))

Then we can get the flight data that's been provided

In [3]:
flightData = pg.get_test_data(300000)
flightData['fl_date'] = pd.to_datetime(flightData['fl_date'])
flightData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,target
0,2018-04-18,DL,DL,DL,1733,DL,N941DL,1733,10397,ATL,...,11481,ECP,"Panama City, FL",1110,1117,N,67.0,1.0,240.0,5.0
1,2018-04-18,DL,DL,DL,1864,DL,N3763D,1864,14679,SAN,...,13487,MSP,"Minneapolis, MN",630,1210,N,220.0,1.0,1532.0,-11.0
2,2018-04-18,DL,DL,DL,1867,DL,N940AT,1867,10781,BTR,...,10397,ATL,"Atlanta, GA",1446,1720,N,94.0,1.0,448.0,-12.0
3,2018-04-18,DL,DL,DL,1875,DL,N327NW,1875,12953,LGA,...,15304,TPA,"Tampa, FL",1145,1451,N,186.0,1.0,1010.0,-28.0
4,2018-04-18,DL,DL,DL,1886,DL,N831DN,1886,10397,ATL,...,12953,LGA,"New York, NY",1630,1851,N,141.0,1.0,762.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293365,2018-04-18,DL,DL,DL,1453,DL,N932AT,1453,13930,ORD,...,11433,DTW,"Detroit, MI",1652,1917,N,85.0,1.0,235.0,-2.0
293366,2018-04-18,DL,DL,DL,1495,DL,N960AT,1495,14869,SLC,...,14908,SNA,"Santa Ana, CA",1706,1755,N,109.0,1.0,588.0,2.0
293367,2018-04-18,DL,DL,DL,1607,DL,N918DL,1607,10397,ATL,...,10994,CHS,"Charleston, SC",1932,2045,N,73.0,1.0,259.0,-3.0
293368,2018-04-18,DL,DL,DL,1640,DL,N591NW,1640,11433,DTW,...,10397,ATL,"Atlanta, GA",600,804,N,124.0,1.0,594.0,-18.0


Add the columns which we want to include directly

In [4]:
modelData = flightData[
    ['crs_dep_time', 'crs_elapsed_time', 'distance', 'target', 'op_unique_carrier', 'fl_date', 'tail_num']
].copy()
modelData['fl_date'] = modelData.fl_date.dt.month
modelData

,crs_dep_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num
0,1110,67.0,240.0,5.0,DL,4,N941DL
1,630,220.0,1532.0,-11.0,DL,4,N3763D
2,1446,94.0,448.0,-12.0,DL,4,N940AT
3,1145,186.0,1010.0,-28.0,DL,4,N327NW
4,1630,141.0,762.0,-4.0,DL,4,N831DN
...,...,...,...,...,...,...,...
293365,1652,85.0,235.0,-2.0,DL,4,N932AT
293366,1706,109.0,588.0,2.0,DL,4,N960AT
293367,1932,73.0,259.0,-3.0,DL,4,N918DL
293368,600,124.0,594.0,-18.0,DL,4,N591NW


Get the expected delay for the city pair

In [5]:
def get_airport_flight_delay(r):
    if r.origin_airport_id not in routeDelay.index:
        return 0
    startCityDelays = routeDelay.loc[r.origin_airport_id]
    destCity = str(r.dest_airport_id)
    if destCity not in startCityDelays:
        return 0
    return startCityDelays[destCity]
# There must be a way to vectorize this
modelData['route_delay'] = flightData.apply(
    get_airport_flight_delay,
    axis=1
)
modelData

,crs_dep_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,route_delay
0,1110,67.0,240.0,5.0,DL,4,N941DL,7.737676
1,630,220.0,1532.0,-11.0,DL,4,N3763D,-5.636418
2,1446,94.0,448.0,-12.0,DL,4,N940AT,4.074303
3,1145,186.0,1010.0,-28.0,DL,4,N327NW,3.355924
4,1630,141.0,762.0,-4.0,DL,4,N831DN,7.392199
...,...,...,...,...,...,...,...,...
293365,1652,85.0,235.0,-2.0,DL,4,N932AT,9.197370
293366,1706,109.0,588.0,2.0,DL,4,N960AT,-0.678944
293367,1932,73.0,259.0,-3.0,DL,4,N918DL,-2.118246
293368,600,124.0,594.0,-18.0,DL,4,N591NW,-0.241674


In [6]:
# Verify there aren't any nulls that got introduced in the last step
modelData.isna().sum()

crs_dep_time         0
crs_elapsed_time     0
distance             0
target               0
op_unique_carrier    0
fl_date              0
tail_num             0
route_delay          0
dtype: int64

Get the carrier's normal delay for this month

In [7]:
modelData['carrier_delay'] = pd.merge(
    modelData, carrierDelay,
    left_on = ['op_unique_carrier', 'fl_date'],
    right_index = True
)['mean_delay']
modelData

,crs_dep_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,route_delay,carrier_delay
0,1110,67.0,240.0,5.0,DL,4,N941DL,7.737676,0.163906
1,630,220.0,1532.0,-11.0,DL,4,N3763D,-5.636418,0.163906
2,1446,94.0,448.0,-12.0,DL,4,N940AT,4.074303,0.163906
3,1145,186.0,1010.0,-28.0,DL,4,N327NW,3.355924,0.163906
4,1630,141.0,762.0,-4.0,DL,4,N831DN,7.392199,0.163906
...,...,...,...,...,...,...,...,...,...
293365,1652,85.0,235.0,-2.0,DL,4,N932AT,9.197370,0.163906
293366,1706,109.0,588.0,2.0,DL,4,N960AT,-0.678944,0.163906
293367,1932,73.0,259.0,-3.0,DL,4,N918DL,-2.118246,0.163906
293368,600,124.0,594.0,-18.0,DL,4,N591NW,-0.241674,0.163906


Get the plane's normal speed based on its tail number

In [8]:
modelData['plane_speed'] = pd.merge(
    modelData, planeSpeed,
    left_on = 'tail_num',
    right_index = True
)['speed']
modelData

,crs_dep_time,crs_elapsed_time,distance,target,op_unique_carrier,fl_date,tail_num,route_delay,carrier_delay,plane_speed
0,1110,67.0,240.0,5.0,DL,4,N941DL,7.737676,0.163906,0.157680
1,630,220.0,1532.0,-11.0,DL,4,N3763D,-5.636418,0.163906,0.138500
2,1446,94.0,448.0,-12.0,DL,4,N940AT,4.074303,0.163906,0.158422
3,1145,186.0,1010.0,-28.0,DL,4,N327NW,3.355924,0.163906,0.141428
4,1630,141.0,762.0,-4.0,DL,4,N831DN,7.392199,0.163906,0.136958
...,...,...,...,...,...,...,...,...,...,...
293365,1652,85.0,235.0,-2.0,DL,4,N932AT,9.197370,0.163906,0.158055
293366,1706,109.0,588.0,2.0,DL,4,N960AT,-0.678944,0.163906,0.158788
293367,1932,73.0,259.0,-3.0,DL,4,N918DL,-2.118246,0.163906,0.159541
293368,600,124.0,594.0,-18.0,DL,4,N591NW,-0.241674,0.163906,0.126754


Now lets veryify all rows are still present, and we haven't added any nulls

In [9]:
modelData.shape, flightData.shape

((293370, 10), (293370, 21))

## Training Models

First lets split the data into training and testing sets

In [10]:
y = modelData['target']
X = modelData.drop(columns=['target', 'tail_num', 'fl_date', 'op_unique_carrier'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
Xtrain.shape, Xtest.shape

((220027, 6), (73343, 6))

There are a couple of models I'd like to try out here

The first one is a basic LinearRegression

In [11]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(Xtrain, ytrain)
# Score gives r-squared, which is what we actually wanted to use as our main
# scoring metric. Don't worry at this stage about actually making predictions
lr.score(Xtest, ytest)

0.024936648030485897

The second model I want to try is XGBoost

In [35]:
import xgboost as xgb
from sklearn.metrics import r2_score

xgreg = xgb.XGBRegressor(
    max_depth = 6,
    n_estimators = 10
)
xgreg.fit(Xtrain, ytrain)
pred = xgreg.predict(Xtest)

r2_score(ytest, pred)

0.025342040275612043

Reducing the number of estimators actually has been improving my results. Does this mean there's a fundamental error with the features I'm trying to use?

We can also try the statsmodels linear regression

In [24]:
import statsmodels.api as sm

trainConst = sm.add_constant(Xtrain)
testConst = sm.add_constant(Xtest)

smreg = sm.OLS(ytrain, trainConst)
res = smreg.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     882.0
Date:                Wed, 22 Sep 2021   Prob (F-statistic):               0.00
Time:                        12:29:15   Log-Likelihood:            -1.1748e+06
No. Observations:              220027   AIC:                         2.350e+06
Df Residuals:                  220020   BIC:                         2.350e+06
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -14.0560      1.168    -12.035      0.000     -16.345     -11.767
crs_dep_time         0.0093      0.000     42.296      0.000       0.009       0.010
crs_elapsed_time    -0.0212      0.008     -2.728      0.006      -0.036      -0.006
distance             0.0024      0.001      2.502      0.012       0.001       0.004
route_delay          0.6938      0.022     31.150      0.000       0.650       0.737
carrier_delay        0.8656      0.022     38.835      0.000       0.822       0.909
plane_speed          0.1985      6.383      0.031      0.975     -12.312      12.710
==============================================================================
Omnibus:                   323596.531   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        195230731.295
Skew:                           8.854   Prob(JB):                         0.00
Kurtosis:                     147.851   Cond. No.                     9.78e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.78e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(Xtrain, ytrain)

rfr.score(Xtest, ytest)

-0.004287691274382777

In [36]:
from sklearn.tree import DecisionTreeRegressor

treeReg = DecisionTreeRegressor()
treeReg.fit(Xtrain, ytrain)

treeReg.score(Xtest, ytest)

-1.1684167336009095

Well these are just awful

The last one I'm going to try in this section is GLM from last week

In [43]:
import statsmodels.api as sm

trainConst = sm.add_constant(Xtrain)
testConst = sm.add_constant(Xtest)

# Try Gamma(err), Gaussian(0.025), Tweedie(err), and Poisson(-0.02)
glm = sm.GLM(ytrain, trainConst, family=sm.families.Gaussian())
res = glm.fit()

pred = res.predict(testConst)

r2_score(ytest, pred)

0.024936648030485675

### Polynomial features

In [46]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
Xpoly = poly.fit_transform(X)

polyTrain, polyTest, polyytrain, polyytest = train_test_split(Xpoly, y)

# The best performing model from earlier was actually the basic regression
lr = LinearRegression()
lr.fit(polyTrain, polyytrain)
# Score gives r-squared, which is what we actually wanted to use as our main
# scoring metric. Don't worry at this stage about actually making predictions
lr.score(polyTest, polyytest)

0.025362747350572534

That is marginally better